### Import Package

In [1]:
import requests
import pandas as pd
import time
import calendar

from bs4 import BeautifulSoup

### Timer Decorate

In [2]:
def timer(fn):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = fn(*args, **kwargs)
        end_time = time.time()
        print("{fn_name} : {time} ms".format(fn_name = fn.__name__, time = end_time - start_time))
        return result
    return wrapper 

### Category Dictionary

In [3]:
category_dict = {
    "100":950203, # 정치
    "101":949986, # 경제
    "102":949987, # 사회
    "103":949988, # 생활/문화
    "104":949990, # 세계
    "105":949984, # IT/과학
}

### Crawling Last Page

In [4]:
# @timer
def last_page(category, date):
    compnentId = category_dict[str(category)]
    url = "http://news.naver.com/main/mainNews.nhn?componentId=" + str(compnentId) + "&date=" + date + " 00:00:00&page=100"
    response = requests.get(url)
    return response.json()["pagerInfo"]["page"]
    
# last_page(100, "2016-06-10")

### Crawling Content, Comment, LikeIt 

In [5]:
# using json
# @timer
def get_likeit(aid, oid):    
    url = "http://news.like.naver.com/likeIt/likeItContent.jsonp?_callback=window.__jindo2_callback._7105&serviceId=NEWS&displayId=NEWS&contentsId=ne_" + str(oid) + "_" + str(aid) + "&lang=ko&viewType=recommend"
    response = requests.get(url)
    return response.text.split('likeItCount":')[1].split(",")[0]
    
# using bs4
# @timer
def get_content(path):
    
    response = requests.get(path)
    dom = BeautifulSoup(response.content, "html.parser")

    if len(dom.select("#articleTitleCommentCount .lo_txt")) == 0:
        return 0, 0, "-"
    
    comment = dom.select_one("#articleTitleCommentCount .lo_txt").text
    content = dom.select_one("#articleBodyContents").text.replace("\n","").replace("\r","").replace("\t","")
    aid = path.split("aid=")[1]
    oid = path.split("oid=")[1].split("&")[0]
    likeit = get_likeit(aid, oid)
    
    return comment, likeit, content

# url = "http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=003&aid=0007327243"
# content_data = get_content(url)
# content_data[0], content_data[1], len(content_data[2])

### Crawling 1 category, 1 day, 1 page

In [7]:
@timer
def one_page_df(category, date, page):
    """ excute time about 5 ~ 6 sec """

    url = "http://news.naver.com/main/mainNews.nhn?componentId=" + str(category_dict[str(category)]) + "&date=" + date + " 00:00:00&page=" + str(page)
    response = requests.get(url)
    article_list = response.json()["itemList"]
    
    result_df = pd.DataFrame(columns=["newsid", "oid", "newspaper", "title", "link", "comment", "likeit", "content", "date", "category"])

    for article in article_list:
        link = "http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=" + str(category) + "&oid=" + article["officeId"] + "&aid=" + article["articleId"]        
        comment, likeit, content = get_content(link)
        
        tmp_dict = {
            "newsid": article["articleId"],
            "oid": article["officeId"],
            "newspaper": article["officeName"],
            "title": article["title"],
            "link": link,
            "comment": comment,
            "likeit": likeit,
            "content": content,
            "date": date,
            "category": str(category-100),
        }
        
        result_df.loc[len(result_df)] = tmp_dict
        
    return result_df

df = one_page_df(105, "2016-07-07", 2)
df

one_page_df : 4.94457483291626 ms


,newsid,oid,newspaper,title,link,comment,likeit,content,date,category
0,0003708082,008,머니투데이,"SKT-CJ헬로비전, 공정위에 의견접수 연장 요청(상보)",http://news.naver.com/main/read.nhn?mode=LSD&m...,13,5,"[머니투데이 이하늘 기자, 진달래 기자] [공정위 전원회의 내달 초로 늦춰질 수도…...",2016-07-07,5
1,0003674447,014,파이낸셜뉴스,[투자활성화 대책] VR테마파크 등 5대 신기술에 600억 지원.. 400억 펀드로...,http://news.naver.com/main/read.nhn?mode=LSD&m...,1,0,VR 투자활성화 정부와 민간이 내년까지 가상현실(VR) 분야에 총 1000억원을 투...,2016-07-07,5
2,0008526843,001,연합뉴스,'로봇 배달부가 왔습니다'…유럽서 음식배달 자율주행로봇 등장,http://news.naver.com/main/read.nhn?mode=LSD&m...,4,1,[저스트잇 홈페이지 캡처] (서울=연합뉴스) 김경윤 기자 = 이제 유럽에서 피...,2016-07-07,5
3,0001080251,016,헤럴드경제,[바람난과학] ‘새로운 파란색’ 발견…이게 무슨 말인가요?,http://news.naver.com/main/read.nhn?mode=LSD&m...,7,12,[HOOC=이정아 기자] Q. 타이거(tiger@)님이 보내주신 질문입니다. 며칠...,2016-07-07,5
4,0002500574,030,전자신문,"[삼성 깜짝실적]갤S7 효과 지속, IM 영업이익 4조원대 복귀",http://news.naver.com/main/read.nhn?mode=LSD&m...,2,1,삼성전자 정보기술·모바일(IM) 부문 2분기 영업이익이 2년 만에 4조원대에 복귀했...,2016-07-07,5
5,0002500566,030,전자신문,"카카오, 버스 지하철 서비스 개편 뒤 이용자 20% 증가",http://news.naver.com/main/read.nhn?mode=LSD&m...,2,1,"카카오버스 서비스 화면<전자신문DB>카카오 버스, 지하철 앱 이용자 수가 급증했다....",2016-07-07,5
6,0002500567,030,전자신문,[삼성 깜짝실적]DS부문 영업익 2.5조 추정 `선방`…고맙다 3D 낸드·OLED,http://news.naver.com/main/read.nhn?mode=LSD&m...,1,0,삼성전자 디바이스솔루션(DS) 부문은 3D 낸드플래시와 유기발광다이오드(OLED) ...,2016-07-07,5
7,0002500568,030,전자신문,우리 회사 도메인이 보내온 메일 무심코 눌렀더니… 변종 랜섬웨어 습격,http://news.naver.com/main/read.nhn?mode=LSD&m...,1,2,#회사에 출근한 직장인 A씨는 간밤에 들어온 이메일을 확인하다 회사 메일 계정과 같...,2016-07-07,5
8,0000380828,031,아이뉴스24,월마트의 애플페이 대항마 美전역 서비스 확대,http://news.naver.com/main/read.nhn?mode=LSD&m...,0,0,<아이뉴스24>[안희권기자] 세계 최대 대형할인매장 월마트가 애플페이 대항마를 미국...,2016-07-07,5
9,0003674400,014,파이낸셜뉴스,케이블TV 뿔났다..&quot;정책일관성 상실&quot; 공정위에 공개질의,http://news.naver.com/main/read.nhn?mode=LSD&m...,4,1,SK텔레콤과 CJ헬로비전의 인수합병(M&A) 불발 위기로 시장 재편의 활로가 막힌...,2016-07-07,5


### 1 category, 1 day, all page

In [14]:
@timer
def one_day_df(category, date):
    """ excute time about 60 sec / 10 page """
    
    last_page_number = int(last_page(category, date))
    
    print("last page : {} / {} / {}".format(last_page_number, category, date))
    
    df_list = []
    
    for page in range(1, last_page_number + 1):
        df = one_page_df(category, date, page)
        df_list.append(df)
        time.sleep(1)
        
    return pd.concat(df_list).reset_index(drop=True)

# day_df = one_day_df(100, "2016-07-06")
# len(day_df)

In [12]:
def day_news(date):

    df_list = []

    for category in range(100, 106):
        day_df = one_day_df(category, date)
        df_list.append(day_df)
    
    return pd.concat(df_list).reset_index(drop=True)

In [13]:
date = "2016-07-06"
df = day_news(date)
df.to_csv("./news/" + date + ".csv", index=False, encoding="utf-8" )

last page : 18 / 100 / 2016-07-07
one_page_df : 6.126049995422363 ms
0     0003074039
1     0003074040
2     0003074038
3     0002155349
4     0003074034
5     0003763894
6     0003074033
7     0003074032
8     0002155344
9     0007337157
10    0000081760
11    0003074031
12    0003621646
13    0002155338
14    0002140539
15    0003074027
16    0003621636
17    0002155336
18    0008527063
19    0003582345
20    0002155329
21    0003191140
22    0003074024
23    0008527043
24    0008527041
Name: newsid, dtype: object
one_page_df : 5.993738889694214 ms
0     0008527037
1     0008527036
2     0008527032
3     0002712605
4     0003783274
5     0001080289
6     0008527025
7     0002155296
8     0003763876
9     0003708144
10    0002155287
11    0000870997
12    0003763874
13    0003763875
14    0003708143
15    0000916437
16    0008526985
17    0002848144
18    0000052094
19    0008526970
20    0003582264
21    0002735708
22    0003783271
23    0003708139
24    0007337040
Name: newsid, dtyp